## 7.1 计算字段
&emsp;&emsp;存储在数据库表中的数据一般不是应用程序所需要的格式，我们需要直接从数据库中检索出转换、计算或格式化过的数据，而不是检索出数据，然后再在客户端应用程序中重新格式化。这就需要用上计算字段，计算字段并不实际存在于数据库表中，而是运行时在SELECT 语句中创建的。

>**字段（field）**：基本上与列（column）的意思相同，经常互换使用，不过数据库列一般称为列，而术语字段通常与计算字段一起使用。

>**提示**：在SQL 语句内可完成的许多转换和格式化工作都可以直接在客户端应用程序内完成。但一般来说，在数据库服务器上完成这些操作比在客户端中完成要快得多。

## 7.2 拼接字段
&emsp;&emsp;下面通过计算字段创建由两列组成的标题：

>**拼接（concatenate）**：将值联结到一起（将一个值附加到另一个值）构成单个值。

In [1]:
import sqlite3 as sql 
import pandas as pd

def select_sql(query):
    conn = sql.connect('tysql.sqlite')
    cur = conn.cursor()
    df = pd.read_sql(query, con=conn)
    cur.close()
    conn.close()
    print(df)

In [2]:
# 从Vendors 表中拼接vend_name, vend_country列并返回，按vend_name排序
query = '''
SELECT vend_name || ' (' || vend_country || ')'
FROM Vendors
ORDER BY vend_name;
'''
select_sql(query)

  vend_name || ' (' || vend_country || ')'
0                      Bear Emporium (USA)
1                         Bears R Us (USA)
2                    Doll House Inc. (USA)
3                  Fun and Games (England)
4                       Furball Inc. (USA)
5                  Jouets et ours (France)


>**说明**：Access 和SQL Server 使用+号。DB2、Oracle、PostgreSQL、SQLite 和Open Office Base 使用||。详细请参阅具体的DBMS 文档。

In [7]:
# 从Vendors 表中拼接vend_name, vend_country列并返回，按vend_name排序
# RTRIM()函数去掉值右边的所有空格
query = '''
SELECT RTRIM(vend_name) || ' (' || RTRIM(vend_country) || ')'
FROM Vendors
ORDER BY vend_name;
'''
select_sql(query)

  RTRIM(vend_name) || ' (' || RTRIM(vend_country) || ')'
0                                Bear Emporium (USA)    
1                                   Bears R Us (USA)    
2                              Doll House Inc. (USA)    
3                            Fun and Games (England)    
4                                 Furball Inc. (USA)    
5                            Jouets et ours (France)    


>**说明**：大多数DBMS 都支持RTRIM()（正如刚才所见，它去掉字符串右边的空格）、LTRIM()（去掉字符串左边的空格）以及TRIM()（去掉字符串左右两边的空格）。

In [8]:
# 使用别名
query = '''
SELECT RTRIM(vend_name) || ' (' || RTRIM(vend_country) || ')' AS vend_title
FROM Vendors
ORDER BY vend_name;
'''
select_sql(query)

                vend_title
0      Bear Emporium (USA)
1         Bears R Us (USA)
2    Doll House Inc. (USA)
3  Fun and Games (England)
4       Furball Inc. (USA)
5  Jouets et ours (France)


>**说明**：别名还有其他用途。常见的用途包括在实际的表列名包含不合法的字符（如空格）时重新命名它，在原来的名字含混或容易误解时扩充它。

## 7.3 执行算术计算
&emsp;&emsp;计算字段的另一常见用途是对检索出的数据进行算术计算：

In [9]:
# 检索订单号为20008 的订单中的所有物品
query = '''
SELECT prod_id, quantity, item_price
FROM OrderItems
WHERE order_num = 20008;
'''
select_sql(query)

  prod_id  quantity  item_price
0  RGAN01         5        4.99
1    BR03         5       11.99
2  BNBG01        10        3.49
3  BNBG02        10        3.49
4  BNBG03        10        3.49


In [10]:
# 检索订单号为20008 的订单中的所有物品
# 并汇总物品的价格（单价乘以订购数量）
query = '''
SELECT prod_id,
    quantity,
    item_price,
    quantity*item_price AS expanded_price
FROM OrderItems
WHERE order_num = 20008;
'''
select_sql(query)

  prod_id  quantity  item_price  expanded_price
0  RGAN01         5        4.99           24.95
1    BR03         5       11.99           59.95
2  BNBG01        10        3.49           34.90
3  BNBG02        10        3.49           34.90
4  BNBG03        10        3.49           34.90


## 7.4 小结
&emsp;&emsp;本章学习了计算字段以及如何创建计算字段，说明了计算字段在字符串拼接和算术计算中的用途。此外，还讲述了如何创建和使用别名。